In [4]:
import arxiv

In [5]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


In [6]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
tool=WikipediaQueryRun(api_wrapper=api_wrapper)
tool.name

'wikipedia'

In [7]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

In [8]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)


In [9]:
vectordb=FAISS.from_documents(documents,OllamaEmbeddings())


In [10]:
retriver=vectordb.as_retriever()


In [ ]:
Input to ChatPromptTemplate is missing variables {'tools', 'tool_names'}.  
Expected: ['agent_scratchpad', 'input', 'tool_names', 'tools'] 
Received: ['input', 'intermediate_steps', 'agent_scratchpad']"


In [11]:
from langchain.tools.retriever import create_retriever_tool
retrival_tool=create_retriever_tool(retriver,"langsmith_search",
                      "Search for information about LangSmith, for any question on langSmith use this tool")




In [44]:


from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant, Please respond to user questions related to langsmith"),
        ("user","Question:{questions}"),
        ("placeholder","{tools}"),
        ("placeholder","{tool_names}")


    ]
)

In [45]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun
#Websites about research

In [46]:
arxiv_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [53]:
tools=[tool,arxiv]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/home/issac/project/envllm/lib/python3.10/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200, arxiv_result=<class 'arxiv.Result'>))]

In [54]:
import os
import configparser
from langchain_community.llms import Ollama
configParser = configparser.RawConfigParser()   
configFilePath = r'/home/issac/project/llm/.env'
configParser.read(configFilePath)

os.environ["LANCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=configParser.get('config','lan_api')
os.environ["LANGCHAIN_PROJECT"]="lan_project 2"
llm=Ollama(model="llama2-uncensored")

In [55]:
from langchain import hub
#prompt=hub.pull("hwchase17/openai-functions-agent")
prompt = hub.pull("hwchase17/react-json")
prompt.messages


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tool_names', 'tools'], template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nThe way you use the tools is by specifying a json blob.\nSpecifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).\n\nThe only values that should be in the "action" field are: {tool_names}\n\nThe $JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. Here is an example of a valid $JSON_BLOB:\n\n```\n{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}\n```\n\nALWAYS use the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction:\n```\n$JSON_BLOB\n```\nObservation: the result of the action\n... (this Thought/Action/Observation can repeat N times)\nThought: I now know the final answer\nF

In [56]:
## Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)



In [57]:
### Agent executor
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)

In [58]:
agent_executor.invoke({"input":"what is langsmith"})



> Entering new AgentExecutor chain...


KeyError: "Input to ChatPromptTemplate is missing variables {'tools', 'tool_names'}.  Expected: ['agent_scratchpad', 'input', 'tool_names', 'tools'] Received: ['input', 'intermediate_steps', 'agent_scratchpad']"

In [ ]:
"""
ChatPromptTemplate error  , tools and tool_names has to be included in 
it may be for all the chat promts"""
    